In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df=pd.read_csv(r'E:/datasets/CollegePlacement.csv')
df.head()

,College_ID,IQ,Prev_Sem_Result,CGPA,Academic_Performance,Internship_Experience,Extra_Curricular_Score,Communication_Skills,Projects_Completed,Placement
0,CLG0030,107,6.61,6.28,8,No,8,8,4,No
1,CLG0061,97,5.52,5.37,8,No,7,8,0,No
2,CLG0036,109,5.36,5.83,9,No,3,1,1,No
3,CLG0055,122,5.47,5.75,6,Yes,1,6,1,No
4,CLG0004,96,7.91,7.69,7,No,8,10,2,No


In [4]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   College_ID              10000 non-null  object 
 1   IQ                      10000 non-null  int64  
 2   Prev_Sem_Result         10000 non-null  float64
 3   CGPA                    10000 non-null  float64
 4   Academic_Performance    10000 non-null  int64  
 5   Internship_Experience   10000 non-null  object 
 6   Extra_Curricular_Score  10000 non-null  int64  
 7   Communication_Skills    10000 non-null  int64  
 8   Projects_Completed      10000 non-null  int64  
 9   Placement               10000 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 781.4+ KB


,IQ,Prev_Sem_Result,CGPA,Academic_Performance,Extra_Curricular_Score,Communication_Skills,Projects_Completed
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,99.471800,7.535673,7.532379,5.546400,4.970900,5.561800,2.513400
std,15.053101,1.447519,1.470141,2.873477,3.160103,2.900866,1.715959
min,41.000000,5.000000,4.540000,1.000000,0.000000,1.000000,0.000000
25%,89.000000,6.290000,6.290000,3.000000,2.000000,3.000000,1.000000
50%,99.000000,7.560000,7.550000,6.000000,5.000000,6.000000,3.000000
75%,110.000000,8.790000,8.770000,8.000000,8.000000,8.000000,4.000000
max,158.000000,10.000000,10.460000,10.000000,10.000000,10.000000,5.000000


In [4]:
df["Placement"]=np.where(df["Placement"]=="Yes",1,0)
df.head()
df.describe()


,IQ,Prev_Sem_Result,CGPA,Academic_Performance,Extra_Curricular_Score,Communication_Skills,Projects_Completed,Placement
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,99.471800,7.535673,7.532379,5.546400,4.970900,5.561800,2.513400,0.165900
std,15.053101,1.447519,1.470141,2.873477,3.160103,2.900866,1.715959,0.372009
min,41.000000,5.000000,4.540000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,89.000000,6.290000,6.290000,3.000000,2.000000,3.000000,1.000000,0.000000
50%,99.000000,7.560000,7.550000,6.000000,5.000000,6.000000,3.000000,0.000000
75%,110.000000,8.790000,8.770000,8.000000,8.000000,8.000000,4.000000,0.000000
max,158.000000,10.000000,10.460000,10.000000,10.000000,10.000000,5.000000,1.000000


In [6]:
df.isnull().sum()



College_ID                0
IQ                        0
Prev_Sem_Result           0
CGPA                      0
Academic_Performance      0
Internship_Experience     0
Extra_Curricular_Score    0
Communication_Skills      0
Projects_Completed        0
Placement                 0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
# Plotly: Distribution of CGPA by Placement Status (overlay + marginal box)
# Ensure a readable placement label exists
label_map = {0: 'Not Placed', 1: 'Placed'}
if 'PlacementLabel' not in df.columns:
    if df['Placement'].dtype != 'O':
        df['PlacementLabel'] = df['Placement'].map(label_map).fillna(df['Placement'].astype(str))
    else:
        df['PlacementLabel'] = df['Placement']

fig = px.histogram(
    df,
    x='CGPA',
    color='PlacementLabel',
    nbins=30,
    barmode='overlay',
    marginal='box',
    opacity=0.55,
    labels={'CGPA': 'CGPA', 'PlacementLabel': 'Placement Status'},
    title='Distribution of CGPA by Placement Status'
)
fig.update_layout(bargap=0.05, legend_title_text='Placement Status')
fig.show()

In [9]:
# Robust aggregation of metrics by Placement (handles missing columns and dtypes)
# Choose your preferred aggregation: 'sum' or 'mean'
agg_method = 'mean'  # change to 'sum' if you prefer totals

# Candidate metric names seen in different datasets
desired_metrics = [
    'Internship',                # common name
    'Internship_Experience',     # alternate name
    'Extra_Curricular_Score',
    'Communication_Skills',
    'Academic_Performance'       # optional, only if present
]

# Keep only columns that actually exist in df
available_metrics = [c for c in desired_metrics if c in df.columns]
if not available_metrics:
    raise KeyError(
        f"None of the expected metric columns were found.\n"
        f"Expected any of: {desired_metrics}\n"
        f"Available columns: {list(df.columns)}"
    )

# Ensure numeric dtype (coerce errors to NaN to avoid aggregation failures)
for col in available_metrics:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Create readable labels for Placement if it's encoded as 0/1
if 'Placement' not in df.columns:
    raise KeyError("Column 'Placement' not found in dataframe.")

if df['Placement'].dtype != 'O':  # numeric codes like 0/1
    label_map = {0: 'Not Placed', 1: 'Placed'}
    df['PlacementLabel'] = df['Placement'].map(label_map).fillna(df['Placement'].astype(str))
    group_key = 'PlacementLabel'
else:
    group_key = 'Placement'

# Primary aggregation
grpp = df.groupby(group_key)[available_metrics].agg(agg_method)

# Also compute both for comparison
grpp_sum = df.groupby(group_key)[available_metrics].sum(numeric_only=True)
grpp_mean = df.groupby(group_key)[available_metrics].mean(numeric_only=True).round(2)

# print(f"Aggregation method used: {agg_method}")
# print(f"Metrics used: {available_metrics}\n")

# print("Result (primary):")
# print(grpp)

# print("\nSums by Placement:")
# print(grpp_sum)

# print("\nMeans by Placement:")
# print(grpp_mean)

In [10]:
# Plotly: Grouped bar charts and summary table for aggregated metrics by Placement
import plotly.express as px
import plotly.graph_objects as go

# Ensure grouped results exist
try:
    _ = grpp, grpp_sum, grpp_mean
except NameError:
    raise RuntimeError("Please run the aggregation cell above before plotting.")

# Reset index for plotting
primary_df = grpp.reset_index()
sum_df = grpp_sum.reset_index()
mean_df = grpp_mean.reset_index()

# Identify the group key column name (first column after reset_index)
group_col = primary_df.columns[0]

# Helper to melt wide to long
def to_long(df):
    return df.melt(id_vars=group_col, var_name='Metric', value_name='Value')

primary_long = to_long(primary_df)
mean_long = to_long(mean_df)
sum_long = to_long(sum_df)

# Main figure: primary aggregation choice
fig_primary = px.bar(
    primary_long,
    x=group_col,
    y='Value',
    color='Metric',
    barmode='group',
    title=f'{agg_method.capitalize()} by Placement',
    labels={group_col: 'Placement', 'Value': agg_method.capitalize()}
)
fig_primary.update_layout(legend_title_text='Metric')
fig_primary.show()

# Means figure
fig_mean = px.bar(
    mean_long,
    x=group_col,
    y='Value',
    color='Metric',
    barmode='group',
    title='Means by Placement',
    labels={group_col: 'Placement', 'Value': 'Mean'}
)
fig_mean.update_layout(legend_title_text='Metric')
fig_mean.show()

# Sums figure
fig_sum = px.bar(
    sum_long,
    x=group_col,
    y='Value',
    color='Metric',
    barmode='group',
    title='Totals by Placement',
    labels={group_col: 'Placement', 'Value': 'Total'}
)
fig_sum.update_layout(legend_title_text='Metric')
fig_sum.show()

# Summary table: means per metric per group and delta
mean_wide = mean_df.set_index(group_col).T  # rows: Metric, columns: groups
mean_wide.index.name = 'Metric'

# Try to identify positive/negative group names for delta
pos_candidates = ['Placed', 'Yes', 1, '1']
neg_candidates = ['Not Placed', 'No', 0, '0']
cols = list(mean_wide.columns)
pos_col = next((c for c in cols if c in pos_candidates), None)
neg_col = next((c for c in cols if c in neg_candidates), None)

table_df = mean_wide.copy()
if pos_col and neg_col:
    table_df['Delta (Placed - Not Placed)'] = table_df[pos_col] - table_df[neg_col]

# Build Plotly table
header_vals = [table_df.index.name or 'Metric'] + [str(c) for c in table_df.columns]
cell_vals = [table_df.index.tolist()] + [table_df[c].round(3).tolist() for c in table_df.columns]

table_fig = go.Figure(data=[go.Table(
    header=dict(values=header_vals, fill_color='#f2f2f2', align='left'),
    cells=dict(values=cell_vals, align='left')
)])
table_fig.update_layout(title='Mean Summary by Placement (with Delta)')
table_fig.show()

In [11]:
## Pre-ML checks and setup

# We’ll quickly validate data quality, inspect class balance, check correlations and mutual information, run a light leakage/multicollinearity screen, and prepare a stratified train/test split. This sets us up for clean modeling in scikit-learn next.

In [12]:
# 1) Data quality snapshot
print('Shape:', df.shape)
print('\nDtypes:\n', df.dtypes)
print('\nMissing values per column:\n', df.isna().sum())

# Basic numeric summary
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
df[numeric_cols].describe().T

Shape: (10000, 11)

Dtypes:
 College_ID                 object
IQ                          int64
Prev_Sem_Result           float64
CGPA                      float64
Academic_Performance        int64
Internship_Experience     float64
Extra_Curricular_Score      int64
Communication_Skills        int64
Projects_Completed          int64
Placement                   int64
PlacementLabel             object
dtype: object

Missing values per column:
 College_ID                    0
IQ                            0
Prev_Sem_Result               0
CGPA                          0
Academic_Performance          0
Internship_Experience     10000
Extra_Curricular_Score        0
Communication_Skills          0
Projects_Completed            0
Placement                     0
PlacementLabel                0
dtype: int64


,count,mean,std,min,25%,50%,75%,max
IQ,10000.0,99.471800,15.053101,41.00,89.00,99.00,110.00,158.00
Prev_Sem_Result,10000.0,7.535673,1.447519,5.00,6.29,7.56,8.79,10.00
CGPA,10000.0,7.532379,1.470141,4.54,6.29,7.55,8.77,10.46
Academic_Performance,10000.0,5.546400,2.873477,1.00,3.00,6.00,8.00,10.00
Internship_Experience,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Extra_Curricular_Score,10000.0,4.970900,3.160103,0.00,2.00,5.00,8.00,10.00
Communication_Skills,10000.0,5.561800,2.900866,1.00,3.00,6.00,8.00,10.00
Projects_Completed,10000.0,2.513400,1.715959,0.00,1.00,3.00,4.00,5.00
Placement,10000.0,0.165900,0.372009,0.00,0.00,0.00,0.00,1.00


In [13]:
# 2) Class balance for target (Placement) with Plotly
label_map = {0: 'Not Placed', 1: 'Placed'}
if df['Placement'].dtype != 'O':
    target_labels = df['Placement'].map(label_map).fillna(df['Placement'].astype(str))
else:
    target_labels = df['Placement']

balance = target_labels.value_counts().rename_axis('Placement').reset_index(name='Count')
balance['Percent'] = (balance['Count'] / balance['Count'].sum() * 100).round(2)

fig = px.bar(balance, x='Placement', y='Count', color='Placement', text='Percent',
             title='Class Balance: Placement', labels={'Count': 'Count'}, height=400)
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(showlegend=False, yaxis_range=[0, balance['Count'].max()*1.15])
fig.show()

balance

,Placement,Count,Percent
0,Not Placed,8341,83.41
1,Placed,1659,16.59


In [14]:
# 3) Numeric correlation heatmap (Plotly)
# Use only numeric columns to avoid errors
import numpy as np
num_df = df.select_dtypes(include=['number']).copy()

# If too many columns, you can limit to a subset of interest
corr = num_df.corr(numeric_only=True)

fig = px.imshow(corr, color_continuous_scale='RdBu', origin='lower',
                title='Correlation Heatmap (numeric features)',
                zmin=-1, zmax=1)
fig.update_layout(height=600)
fig.show()

corr.sort_values(by='Placement', ascending=False)['Placement'] if 'Placement' in corr.columns else corr

Placement                 1.000000
Communication_Skills      0.323155
CGPA                      0.321607
Prev_Sem_Result           0.317896
IQ                        0.285914
Projects_Completed        0.217499
Extra_Curricular_Score   -0.004740
Academic_Performance     -0.014920
Internship_Experience          NaN
Name: Placement, dtype: float64

In [15]:
# 4) Mutual Information (numeric-only quick scan)
from sklearn.feature_selection import mutual_info_classif

X_num = df.select_dtypes(include=['number']).drop(columns=['Placement'], errors='ignore')
y = df['Placement']

# Fill NaN with median; if column is all NaN, fill with 0
X_num_filled = X_num.fillna(X_num.median()).fillna(0)

mi = mutual_info_classif(X_num_filled, y, random_state=42)
mi_series = pd.Series(mi, index=X_num.columns).sort_values(ascending=False)

fig = px.bar(mi_series, title='Mutual Information with Placement')
fig.update_layout(xaxis_title='Feature', yaxis_title='MI Score')
fig.show()

mi_series

Communication_Skills      0.076192
CGPA                      0.070061
IQ                        0.065762
Prev_Sem_Result           0.064915
Projects_Completed        0.043411
Academic_Performance      0.000000
Extra_Curricular_Score    0.000000
Internship_Experience     0.000000
dtype: float64

In [16]:
# 5) Light leakage/collinearity check
# Flag features that correlate too strongly with the target or each other
strong_with_target = corr['Placement'].abs().sort_values(ascending=False).drop(labels=['Placement'], errors='ignore') if 'Placement' in corr.columns else pd.Series(dtype=float)
strong_pairs = (corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
                   .stack()
                   .abs()
                   .sort_values(ascending=False))

print('Top correlations with target (abs):\n', strong_with_target.head(10))
print('\nTop inter-feature correlations (abs):\n', strong_pairs.head(10))

Top correlations with target (abs):
 Communication_Skills      0.323155
CGPA                      0.321607
Prev_Sem_Result           0.317896
IQ                        0.285914
Projects_Completed        0.217499
Academic_Performance      0.014920
Extra_Curricular_Score    0.004740
Internship_Experience          NaN
Name: Placement, dtype: float64

Top inter-feature correlations (abs):
 Prev_Sem_Result       CGPA                    0.980768
Communication_Skills  Placement               0.323155
CGPA                  Placement               0.321607
Prev_Sem_Result       Placement               0.317896
IQ                    Placement               0.285914
Projects_Completed    Placement               0.217499
CGPA                  Projects_Completed      0.020872
Academic_Performance  Communication_Skills    0.019733
Prev_Sem_Result       Projects_Completed      0.018434
CGPA                  Communication_Skills    0.018242
dtype: float64


In [5]:
# 6) Define feature lists for sklearn
numeric_features = df.select_dtypes(include=['number']).columns.tolist()
if 'Placement' in numeric_features:
    numeric_features.remove('Placement')

categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Placement' in categorical_features:
    categorical_features.remove('Placement')

print('Numeric features:', numeric_features)
print('Categorical features:', categorical_features)

Numeric features: ['IQ', 'Prev_Sem_Result', 'CGPA', 'Academic_Performance', 'Extra_Curricular_Score', 'Communication_Skills', 'Projects_Completed']
Categorical features: ['College_ID', 'Internship_Experience']


In [18]:
# 7) Stratified train/validation split (test later when ready)
from sklearn.model_selection import train_test_split

X = df[numeric_features + categorical_features]
y = df['Placement']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print('Train shape:', X_train.shape, ' Val shape:', X_val.shape)
print('Class ratio (train):', y_train.value_counts(normalize=True).round(3).to_dict())
print('Class ratio (val):  ', y_val.value_counts(normalize=True).round(3).to_dict())

Train shape: (8000, 10)  Val shape: (2000, 10)
Class ratio (train): {0: 0.834, 1: 0.166}
Class ratio (val):   {0: 0.834, 1: 0.166}


## Machine Learning Models Training

Now we'll train multiple classification models, evaluate their performance, and compare results.

In [19]:
# Import additional models and metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
import time

# Handle categorical features by encoding them
X_train_processed = X_train.copy()
X_val_processed = X_val.copy()

# Encode categorical features if any exist
if len(categorical_features) > 0:
    le = LabelEncoder()
    for col in categorical_features:
        # Fit on combined data to ensure consistent encoding
        combined = pd.concat([X_train_processed[col], X_val_processed[col]], axis=0)
        le.fit(combined.astype(str))
        X_train_processed[col] = le.transform(X_train_processed[col].astype(str))
        X_val_processed[col] = le.transform(X_val_processed[col].astype(str))

# Handle missing values in numeric features
X_train_filled = X_train_processed.fillna(X_train_processed.median()).fillna(0)
X_val_filled = X_val_processed.fillna(X_val_processed.median()).fillna(0)

# Standardize features (important for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filled)
X_val_scaled = scaler.transform(X_val_filled)

print("Data prepared for training!")
print(f"Training samples: {X_train_scaled.shape[0]}")
print(f"Validation samples: {X_val_scaled.shape[0]}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Categorical features encoded: {len(categorical_features)}")

Data prepared for training!
Training samples: 8000
Validation samples: 2000
Number of features: 10
Categorical features encoded: 2


In [20]:
# Define models to train
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
    'Support Vector Machine': SVC(random_state=42, probability=True),
    'Naive Bayes': GaussianNB()
}

# Dictionary to store results
results = {}

# Train all models
print("Training models...\n")
for name, model in models.items():
    start_time = time.time()
    
    # Train the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_val_scaled)
    y_pred_proba = model.predict_proba(X_val_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, zero_division=0)
    recall = recall_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_val, y_pred_proba) if y_pred_proba is not None else None
    
    training_time = time.time() - start_time
    
    # Store results
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'training_time': training_time,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"✓ {name}")
    print(f"  Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    if roc_auc:
        print(f"  ROC-AUC: {roc_auc:.4f} | Training Time: {training_time:.3f}s")
    print()

print("Training complete!")

Training models...

✓ Logistic Regression
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.027s

✓ K-Nearest Neighbors
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.158s

✓ Decision Tree
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.014s

✓ Random Forest
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.348s

✓ Random Forest
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.348s

✓ Gradient Boosting
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.476s

✓ Gradient Boosting
  Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000
  ROC-AUC: 1.0000 | Training Time: 0.476s

✓ Support Vector Machine
  Accuracy: 1.0000 | Precision: 1.0000 

In [21]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results],
    'Precision': [results[m]['precision'] for m in results],
    'Recall': [results[m]['recall'] for m in results],
    'F1-Score': [results[m]['f1_score'] for m in results],
    'ROC-AUC': [results[m]['roc_auc'] if results[m]['roc_auc'] else 0 for m in results],
    'Training Time (s)': [results[m]['training_time'] for m in results]
}).sort_values('Accuracy', ascending=False)

comparison_df

,Model,Accuracy,Precision,Recall,F1-Score,ROC-AUC,Training Time (s)
0,Logistic Regression,1.0,1.0,1.0,1.0,1.0,0.026975
1,K-Nearest Neighbors,1.0,1.0,1.0,1.0,1.0,0.158343
2,Decision Tree,1.0,1.0,1.0,1.0,1.0,0.014001
3,Random Forest,1.0,1.0,1.0,1.0,1.0,0.347980
4,Gradient Boosting,1.0,1.0,1.0,1.0,1.0,0.475998
5,Support Vector Machine,1.0,1.0,1.0,1.0,1.0,0.406936
6,Naive Bayes,1.0,1.0,1.0,1.0,1.0,0.010977


In [22]:
# Visualize model comparison - Accuracy
fig = px.bar(comparison_df.sort_values('Accuracy', ascending=True), 
             x='Accuracy', 
             y='Model', 
             orientation='h',
             title='Model Comparison - Accuracy',
             text='Accuracy',
             color='Accuracy',
             color_continuous_scale='Viridis')
fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.update_layout(height=500, showlegend=False)
fig.show()

In [24]:
# Visualize all metrics comparison
metrics_df = comparison_df.melt(id_vars='Model', 
                                value_vars=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'],
                                var_name='Metric', 
                                value_name='Score')

fig = px.bar(metrics_df, 
             x='Model', 
             y='Score', 
             color='Metric',
             barmode='group',
             title='Model Performance - All Metrics Comparison',
             labels={'Score': 'Score', 'Model': 'Model'},
             height=500)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [25]:
# Get the best model
best_model_name = comparison_df.iloc[0]['Model']
best_model_results = results[best_model_name]

print(f"🏆 Best Model: {best_model_name}")
print(f"   Accuracy: {best_model_results['accuracy']:.4f}")
print(f"   Precision: {best_model_results['precision']:.4f}")
print(f"   Recall: {best_model_results['recall']:.4f}")
print(f"   F1-Score: {best_model_results['f1_score']:.4f}")
if best_model_results['roc_auc']:
    print(f"   ROC-AUC: {best_model_results['roc_auc']:.4f}")

🏆 Best Model: Logistic Regression
   Accuracy: 1.0000
   Precision: 1.0000
   Recall: 1.0000
   F1-Score: 1.0000
   ROC-AUC: 1.0000


In [26]:
# Confusion Matrix for Best Model
cm = confusion_matrix(y_val, best_model_results['y_pred'])

# Create confusion matrix visualization
fig = px.imshow(cm, 
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['Not Placed', 'Placed'],
                y=['Not Placed', 'Placed'],
                title=f'Confusion Matrix - {best_model_name}',
                color_continuous_scale='Blues',
                text_auto=True)
fig.update_layout(height=500, width=600)
fig.show()

# Print confusion matrix details
print("\nConfusion Matrix Breakdown:")
print(f"True Negatives (Correctly predicted Not Placed): {cm[0, 0]}")
print(f"False Positives (Incorrectly predicted Placed): {cm[0, 1]}")
print(f"False Negatives (Incorrectly predicted Not Placed): {cm[1, 0]}")
print(f"True Positives (Correctly predicted Placed): {cm[1, 1]}")


Confusion Matrix Breakdown:
True Negatives (Correctly predicted Not Placed): 1668
False Positives (Incorrectly predicted Placed): 0
False Negatives (Incorrectly predicted Not Placed): 0
True Positives (Correctly predicted Placed): 332


In [27]:
# Classification Report for Best Model
print(f"Classification Report - {best_model_name}\n")
print(classification_report(y_val, best_model_results['y_pred'], 
                          target_names=['Not Placed', 'Placed']))

Classification Report - Logistic Regression

              precision    recall  f1-score   support

  Not Placed       1.00      1.00      1.00      1668
      Placed       1.00      1.00      1.00       332

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [28]:
# ROC Curves for all models with probability predictions
fig = go.Figure()

for name in results.keys():
    if results[name]['y_pred_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_val, results[name]['y_pred_proba'])
        auc = results[name]['roc_auc']
        fig.add_trace(go.Scatter(x=fpr, y=tpr, 
                                mode='lines',
                                name=f'{name} (AUC={auc:.3f})'))

# Add diagonal reference line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], 
                        mode='lines',
                        name='Random Classifier',
                        line=dict(dash='dash', color='gray')))

fig.update_layout(
    title='ROC Curves - Model Comparison',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=600,
    width=800
)
fig.show()

In [29]:
# Feature Importance (for tree-based models)
feature_importance_models = ['Decision Tree', 'Random Forest', 'Gradient Boosting']

for model_name in feature_importance_models:
    if model_name in results:
        model = results[model_name]['model']
        if hasattr(model, 'feature_importances_'):
            importance_df = pd.DataFrame({
                'Feature': numeric_features + categorical_features,
                'Importance': model.feature_importances_
            }).sort_values('Importance', ascending=False).head(10)
            
            fig = px.bar(importance_df.sort_values('Importance', ascending=True),
                        x='Importance',
                        y='Feature',
                        orientation='h',
                        title=f'Top 10 Feature Importance - {model_name}',
                        color='Importance',
                        color_continuous_scale='Teal')
            fig.update_layout(height=500)
            fig.show()

## Final Conclusions and Recommendations

Summary of the placement prediction ML project:

In [30]:
# Final Summary
print("="*80)
print(" PLACEMENT PREDICTION - ML PROJECT SUMMARY")
print("="*80)
print("\n📊 DATASET OVERVIEW:")
print(f"   • Total samples: {df.shape[0]}")
print(f"   • Features: {df.shape[1] - 1}")
print(f"   • Target: Placement (Binary Classification)")
print(f"   • Train/Val split: 80/20")

print("\n🏆 BEST PERFORMING MODEL:")
print(f"   • Model: {best_model_name}")
print(f"   • Accuracy: {best_model_results['accuracy']:.2%}")
print(f"   • Precision: {best_model_results['precision']:.2%}")
print(f"   • Recall: {best_model_results['recall']:.2%}")
print(f"   • F1-Score: {best_model_results['f1_score']:.2%}")
if best_model_results['roc_auc']:
    print(f"   • ROC-AUC: {best_model_results['roc_auc']:.2%}")

print("\n📈 TOP 3 MODELS BY ACCURACY:")
for idx, row in comparison_df.head(3).iterrows():
    print(f"   {idx+1}. {row['Model']}: {row['Accuracy']:.2%}")

print("\n🔑 KEY INSIGHTS:")
print(f"   • Top features correlated with placement:")
if 'corr' in dir() and 'Placement' in corr.columns:
    top_features = corr['Placement'].abs().sort_values(ascending=False).head(4)[1:]
    for feat, val in top_features.items():
        print(f"     - {feat}: {val:.3f}")

print("\n💡 RECOMMENDATIONS:")
print("   • Use the best model for placement predictions")
print("   • Focus on improving top correlated features for better placement chances")
print("   • Consider ensemble methods for production deployment")
print("   • Regularly retrain with new data for improved accuracy")

print("\n" + "="*80)
print(" PROJECT COMPLETE ✓")
print("="*80)

 PLACEMENT PREDICTION - ML PROJECT SUMMARY

📊 DATASET OVERVIEW:
   • Total samples: 10000
   • Features: 10
   • Target: Placement (Binary Classification)
   • Train/Val split: 80/20

🏆 BEST PERFORMING MODEL:
   • Model: Logistic Regression
   • Accuracy: 100.00%
   • Precision: 100.00%
   • Recall: 100.00%
   • F1-Score: 100.00%
   • ROC-AUC: 100.00%

📈 TOP 3 MODELS BY ACCURACY:
   1. Logistic Regression: 100.00%
   2. K-Nearest Neighbors: 100.00%
   3. Decision Tree: 100.00%

🔑 KEY INSIGHTS:
   • Top features correlated with placement:
     - Communication_Skills: 0.323
     - CGPA: 0.322
     - Prev_Sem_Result: 0.318

💡 RECOMMENDATIONS:
   • Use the best model for placement predictions
   • Focus on improving top correlated features for better placement chances
   • Consider ensemble methods for production deployment
   • Regularly retrain with new data for improved accuracy

 PROJECT COMPLETE ✓


In [1]:
# Export cleaned dataset and related artifacts to Excel
out_path = r'e:\jupy -ipnb\placements_export.xlsx'

# Build dataframes to export
raw_df = df.copy()
cleaned_df = df.copy()  # df already contains cleaned/encoded columns used above

# Prepare train and validation dataframes (combine features + target)
train_df = pd.concat([pd.DataFrame(X_train_filled).reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_df.columns = list(pd.DataFrame(X_train_filled).columns) + ['Placement']

val_df = pd.concat([pd.DataFrame(X_val_filled).reset_index(drop=True), y_val.reset_index(drop=True)], axis=1)
val_df.columns = list(pd.DataFrame(X_val_filled).columns) + ['Placement']

# Create Excel with multiple sheets
try:
    with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
        raw_df.to_excel(writer, sheet_name='raw_data', index=False)
        cleaned_df.to_excel(writer, sheet_name='cleaned_data', index=False)
        train_df.to_excel(writer, sheet_name='train', index=False)
        val_df.to_excel(writer, sheet_name='validation', index=False)
        if 'comparison_df' in globals():
            comparison_df.to_excel(writer, sheet_name='model_comparison', index=False)
    print(f"Export successful: {out_path}")
except Exception as e:
    print('Export failed:', e)

NameError: name 'df' is not defined